In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import pandas as pd
import numpy as np
import xarray as xr
import seaborn as sns
import joblib
import sys
from glob import glob 
import wrf


from sklearn.metrics import r2_score, mean_squared_error
from collections import Counter

sys.path.append('/home/freimax/msc_thesis/scripts/random_forest/')
import rf_functions



sys.path.append('/home/freimax/msc_thesis/scripts/IFS-18/')
import traj_processing_functions


sys.path.append('/home/freimax/msc_thesis/scripts/helpers/')
import helper

sys.path.append('/home/freimax/msc_thesis/scripts/helpers/')
import data_preprocessing_helpers

sys.path.append('/home/freimax/msc_thesis/scripts/plotting_functions/')
import plot_helpers
import make_plots_traj

In [3]:
model_tsubsi, model_tmeltsi, model_tevr = helper.load_rf_models()

                   *** Load Models ***                    
  Load model tsubs
  Load model tmeltsi
  Load model tevr


In [8]:
df_traj = traj_processing_functions.load_trajs_predict_bcp(traj_file_path=f'/net/helium/atmosdyn/freimax/data_msc/casestudy_ra/IFS-17/traj-20170410_16-ID-000019.txt', 
                                                           model_tmeltsi=model_tmeltsi,
                                                           model_tsubsi=model_tsubsi,
                                                           model_tevr=model_tevr,
                                                           folder_branch_list=None, era5_data=False, debug=False)

  Start loading Traj fiels
  Invalid values encountered, dropping them... (trajectories dropped: 64 of 2085)
Done Loading/Cleaning
  Make predictions along trajectory 


In [9]:
## Save the dataframe
save_path = '/net/helium/atmosdyn/freimax/data_msc/casestudy_ra/IFS-17'
df_traj.to_hdf(f'{save_path}/df_big_ifs17.h5', key='df', mode='w')

In [10]:
df_traj.describe()

,time,lon,lat,p,tmeltsi_pred,tsubsi_pred,tevr_pred,Atmeltsi_pred,Atsubsi_pred,Atevr_pred,...,PS,Q,T,RWC,LWC,OMEGA,RH,SIWC,NOBCP_FLAG,id_CYC
count,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,...,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,99029.000000,99029.0,99029.0
mean,-24.000000,157.826358,47.302448,930.410344,-0.014911,-0.077078,-0.000907,-0.266567,-1.642485,-0.030239,...,996.275435,0.002788,-2.732822,0.001662,0.011445,-0.041649,80.023080,0.045550,0.0,1.0
std,14.142207,28.493033,3.163549,76.827086,0.079586,0.131317,0.014103,0.943767,1.819948,0.171784,...,12.827649,0.000984,4.533313,0.010492,0.028378,0.205473,17.407317,0.083081,0.0,0.0
min,-48.000000,-179.990000,32.760000,424.000000,-1.642520,-1.620212,-0.993164,-10.664227,-9.249792,-2.906193,...,885.813000,0.000000,-47.629000,0.000000,0.000000,-1.561000,1.653000,0.000000,0.0,1.0
25%,-36.000000,157.190000,45.490000,904.000000,0.000000,-0.085459,0.000000,-0.000370,-2.531744,0.000000,...,988.048000,0.002000,-4.968000,0.000000,0.000000,-0.075000,72.868000,0.000000,0.0,1.0
50%,-24.000000,161.470000,46.820000,957.000000,0.000000,-0.025659,0.000000,0.000000,-0.977908,0.000000,...,999.852000,0.003000,-1.658000,0.000000,0.000000,-0.009000,84.300000,0.005000,0.0,1.0
75%,-12.000000,164.460000,48.200000,985.000000,0.000000,0.000000,0.000000,0.000000,-0.180798,0.000000,...,1005.973000,0.003000,0.346000,0.000000,0.006000,0.040000,92.169000,0.048000,0.0,1.0
max,0.000000,179.990000,61.280000,1020.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1022.210000,0.010000,15.972000,0.319000,0.330000,1.854000,104.209000,0.623000,0.0,1.0


In [17]:
traj_era_path = '/net/helium/atmosdyn/freimax/data_msc/casestudy_ra/ERA5/trajectories-20170410_16-ID-504663.txt'
df_traj_era = traj_processing_functions.load_trajs_predict_bcp(traj_file_path=traj_era_path, 
                                                           model_tmeltsi=model_tmeltsi,
                                                           model_tsubsi=model_tsubsi,
                                                           model_tevr=model_tevr,
                                                           folder_branch_list=None, era5_data=True, debug=False)

  Start loading Traj fiels
Done Loading/Cleaning
  Make predictions along trajectory 


In [18]:
df_traj_era.describe()

,time,lon,lat,p,tmeltsi_pred,tsubsi_pred,tevr_pred,Atmeltsi_pred,Atsubsi_pred,Atevr_pred,...,PV,LWC,RWC,Q,RH,T,OMEGA,SIWC,NOBCP_FLAG,id_CYC
count,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,...,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,22442.000000,22442.0,22442.0
mean,-24.000000,162.171403,46.648658,947.492336,-0.024949,-0.120903,-0.002399,-0.590239,-2.314287,-0.088172,...,0.553085,0.015848,0.004269,3.196400,84.631528,-1.172223,-0.075803,0.050012,0.0,1.0
std,14.142451,4.212051,2.746808,66.943304,0.113551,0.187551,0.017383,1.380256,2.647798,0.221047,...,0.830262,0.034798,0.015202,0.841061,11.377342,3.652135,0.202981,0.083916,0.0,0.0
min,-48.000000,129.380000,38.390000,591.000000,-1.845090,-1.581987,-0.455079,-7.936695,-11.286231,-1.382750,...,-2.528000,0.000000,0.000000,0.073000,7.259000,-27.840000,-1.404000,0.000000,0.0,1.0
25%,-36.000000,159.730000,44.890000,932.000000,-0.000046,-0.140389,-0.000239,-0.149278,-3.567061,-0.044081,...,-0.002000,0.000000,0.000000,2.718000,76.149000,-2.527750,-0.094000,0.000370,0.0,1.0
50%,-24.000000,162.550000,46.310000,972.000000,0.000000,-0.042284,0.000000,-0.000297,-1.220263,-0.000918,...,0.100000,0.000000,0.000000,3.123000,86.910500,-0.357500,-0.017000,0.005659,0.0,1.0
75%,-12.000000,165.000000,47.880000,992.000000,0.000000,-0.009418,0.000000,0.000000,-0.188981,0.000000,...,0.969000,0.012840,0.000081,3.536000,93.737000,0.831750,0.020000,0.062172,0.0,1.0
max,0.000000,179.290000,58.910000,1016.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.470000,0.368492,0.193816,8.904000,101.756000,14.060000,0.882000,0.504872,0.0,1.0


In [19]:
## Save the dataframe
save_path_era = '/net/helium/atmosdyn/freimax/data_msc/casestudy_ra/ERA5'
df_traj_era.to_hdf(f'{save_path_era}/df_big_era.h5', key='df', mode='w')